# 在 Google Colab 中进行 EchoHeart 的 Qwen 微调

这个 Notebook 会自动配置环境、启动微调 (QLoRA)、运行测试、合并 LoRA 适配器并导出 GGUF 模型。

**配置**：您可以通过修改下面的第一个代码单元格中的变量来指定要使用的基础模型和数据集。

**步骤：**
1. 配置变量、克隆/更新 GitHub 仓库并定义路径。
2. 安装必要的依赖项。
3. 运行训练脚本 (QLoRA)。
4. (可选) 运行测试脚本与微调后的模型(适配器)交互。
5. 合并 LoRA 适配器到基础模型。
6. (可选) 将合并后的模型转换为 GGUF 格式。

In [ ]:
# 1. 配置、克隆/更新仓库并定义路径
import os

# ==============================================================
#                  主要配置区域 (在此处修改)
# ==============================================================

# --- A. 核心配置 ---
# 指定要微调的基础模型 (Hugging Face 名称或路径)
base_model_name: str = "Qwen/Qwen2.5-1.5B-Instruct"
# 指定数据集文件路径 (相对于仓库根目录)
dataset_file: str = "data/converted_dataset.json"

# --- B. 训练超参数 (按需修改） ---
num_train_epochs: int = 2          # 训练轮数
learning_rate: float = 2e-4        # 学习率
weight_decay: float = 0.01         # 权重衰减
max_grad_norm: float = 1.0           # 梯度裁剪范数
seed: int = 42                   # 随机种子

# --- C. LoRA 特定参数 (按需修改) ---
lora_r: int = 16                 # LoRA 秩
lora_alpha: int = 32               # LoRA alpha
lora_dropout: float = 0.05           # LoRA dropout

# --- D. 保存与日志 (按需修改) ---
save_steps: int = 25               # 每 N 步保存一次 checkpoint
logging_steps: int = 5             # 每 N 步记录一次日志

# --- E. 输出目录 (高级，通常自动生成) ---
# 默认会根据 base_model_name 自动生成。
# 如果要自定义，请将 None 替换为你的路径字符串，例如: "output/my_custom_run"
custom_output_dir: str | None = None
# ------------------------------------------

# ==============================================================
#          路径计算和环境设置 (通常无需修改)
# ==============================================================

# --- 确定最终输出目录 ---
# 如果用户没有设置 custom_output_dir，则自动生成
if custom_output_dir:
    adapter_output_dir: str = custom_output_dir
    print(f"[dim]使用自定义适配器输出目录: {adapter_output_dir}[/dim]")
else:
    # 否则，自动生成
    adapter_output_dir: str = f"output/{base_model_name.split('/')[-1]}-qlora-ft"
    print(f"[dim]使用自动生成的适配器输出目录: {adapter_output_dir}[/dim]")

# --- 固定路径和绝对路径计算 ---
repo_path: str = '/content/echoheart_demo'
# 将相对路径转换为绝对路径
adapter_path: str = os.path.join(repo_path, adapter_output_dir)
dataset_abs_path: str = os.path.join(repo_path, dataset_file)
# 其他路径基于最终的 adapter_path
merged_model_output_dir: str = f"{adapter_output_dir.replace('-qlora-ft', '-merged-ft')}"
merged_model_path: str = os.path.join(repo_path, merged_model_output_dir)
gguf_output_file: str = f"{os.path.basename(merged_model_output_dir)}/gguf-model-f16.gguf" # GGUF 文件名放在合并后的目录里
gguf_output_abs_path: str = os.path.join(repo_path, merged_model_output_dir, os.path.basename(gguf_output_file)) # 确保 GGUF 在合并目录内
# -----------------------------

print("\n--- 最终配置信息 ---")
print(f"基础模型: {base_model_name}")
print(f"数据集文件: {dataset_abs_path}")
print(f"适配器输出目录: {adapter_path}")
print(f"学习率: {learning_rate}, Epochs: {num_train_epochs}, LoRA r: {lora_r}")
print(f"合并模型输出目录: {merged_model_path}")
print(f"GGUF 输出文件: {gguf_output_abs_path}")
print("----------------------")

# 克隆或更新仓库
if not os.path.exists(repo_path):
  print(f"\nCloning repository into {repo_path}...")
  # 使用 --depth 1 加快克隆速度，如果需要完整历史则移除
  !git clone --depth 1 https://github.com/shuakami/echoheart_demo.git {repo_path}
else:
  print(f"\nRepository already exists at {repo_path}.")

%cd {repo_path}

print("\nPulling latest changes...")
# 使用 --no-edit 避免潜在的编辑器冲突，并确保拉取成功
try:
    !git pull origin master --no-edit --ff-only
except Exception as e:
     print(f"[yellow]Git pull failed: {e}. Continuing with local version.[/yellow]")


# 确保数据集文件存在
if not os.path.exists(dataset_abs_path):
  print(f"\n[bold red]错误：指定的数据集文件不存在: {dataset_abs_path}[/bold red]")
  # 可以在这里引发错误停止执行
  raise FileNotFoundError(f"Dataset file not found: {dataset_abs_path}")
else:
  print(f"\n数据集文件确认存在: {dataset_abs_path}")

print("\nWorkspace:")
!pwd

In [ ]:
# 2. 安装依赖
print("Installing dependencies from requirements.txt...")
!pip install -q -r requirements.txt

print("Applying dependency fixes...")
!pip install fsspec==2024.12.0

In [ ]:
# 3. 运行训练脚本 (QLoRA)
print("Starting QLoRA training...")

train_command = (
    f"python train.py "
    f"--base_model_name \"{base_model_name}\" "
    f"--dataset_file \"{dataset_file}\" "
    f"--output_dir \"{adapter_path}\" " 
    f"--num_train_epochs {num_train_epochs} "
    f"--learning_rate {learning_rate} "
    f"--weight_decay {weight_decay} "
    f"--max_grad_norm {max_grad_norm} "
    f"--lora_r {lora_r} "
    f"--lora_alpha {lora_alpha} "
    f"--lora_dropout {lora_dropout} "
    f"--save_steps {save_steps} "
    f"--logging_steps {logging_steps} "
    f"--seed {seed}"
)

print("\n--- Running Training Command ---")
print(train_command)
print("------------------------------\n")

# 执行命令
!{train_command}

## 训练完成！

微调后的模型保存在 Colab 环境文件系统的 `output/qwen-ft` 目录中。

In [ ]:
# 4. 运行测试脚本
print("Starting non-interactive testing session...")
!python test_model.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}"

## 5. (可选) 转换为 GGUF 格式

转换后的文件将尝试保存在 `{gguf_output_file}`。

In [ ]:
# 运行 LoRA 合并脚本
print("Starting LoRA merge...")
!python merge_lora.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}" --output_path "{merged_model_path}"

# 转换为 GGUF
print("Starting GGUF conversion...")
!python convert_to_gguf.py --model_dir "{merged_model_path}" --output_file "{gguf_output_abs_path}" --out_type f16

## (可选) 查看 train.py 的高级参数

运行下面的代码单元格可以显示 `train.py` 脚本支持的所有命令行参数及其说明和默认值。
如果您想覆盖默认设置（例如调整学习率、LoRA rank、保存步数等），可以在第 3 步运行训练时手动添加这些参数。

In [ ]:
!python train.py --help